In [ ]:
# Install required packages
%pip install pandas numpy seaborn scikit-learn

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Read the CSV data from the local file
file_path = r"C:\Users\lewis\OneDrive\Documents\2023-02-DSI-WE\Sal1776\Accidental_Drug_Related_Deaths_2012-2021.csv"
df = pd.read_csv(file_path)

# Rename columns with friendly names
df.rename(columns={
    "Date": "Date",
    "Date Type": "Date Type",
    "Age": "Age",
    "Sex": "Gender",
    "Race": "Race",
    "Ethnicity": "Ethnicity",
    "Residence City": "Residence City",
    "Residence County": "Residence County",
    "Residence State": "Residence State",
    "Injury City": "Injury City",
    "Injury County": "Injury County",
    "Injury State": "Injury State",
    "Injury Place": "Injury Place",
    "Description of Injury": "Description of Injury",
    "Death City": "Death City",
    "Death County": "Death County",
    "Death State": "Death State",
    "Location": "Location",
    "Location if Other": "Location if Other",
    "Cause of Death": "Cause of Death",
    "Manner of Death": "Manner of Death",
    "Other Significant Conditions ": "Other Significant Conditions",
    "Heroin": "Heroin",
    "Heroin death certificate (DC)": "Heroin (DC)",
    "Cocaine": "Cocaine",
    "Fentanyl": "Fentanyl",
    "Fentanyl Analogue": "Fentanyl Analogue",
    "Oxycodone": "Oxycodone",
    "Oxymorphone": "Oxymorphone",
    "Ethanol": "Alcohol",
    "Hydrocodone": "Hydrocodone",
    "Benzodiazepine": "Benzodiazepine",
    "Methadone": "Methadone",
    "Meth/Amphetamine": "Meth/Amphetamine",
    "Amphetamines": "Amphetamines",
    "Tramadol": "Tramadol",
    "Hydromorphone": "Hydromorphone",
    "Morphine (Not Heroin)": "Morphine",
    "Xylazine": "Xylazine",
    "Gabapentin": "Gabapentin",
    "Opiate NOS": "Opiate NOS",
    "Heroin/Morph/Codeine": "Heroin/Morph/Codeine",
    "Other Opioid": "Other Opioid",
    "Any Opioid": "Any Opioid",
    "Other": "Other",
    "ResidenceCityGeo": "ResidenceCityGeo",
    "InjuryCityGeo": "InjuryCityGeo",
    "DeathCityGeo": "DeathCityGeo"
}, inplace=True)

# Drop irrelevant columns
df.drop(["Residence County", "Injury County", "Injury State", "Other Significant Conditions", "Heroin (DC)", 
         "Fentanyl Analogue", "Hydromorphone", "Xylazine", "Gabapentin", "Opi
